# K-Means Clustering Method from Kaggle

Source: https://www.kaggle.com/minc33/visualizing-high-dimensional-clusters/notebook

In [ ]:
#Basic imports
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#sklearn imports
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.manifold import TSNE #T-Distributed Stochastic Neighbor Embedding
from sklearn.cluster import KMeans #K-Means Clustering
from sklearn.preprocessing import StandardScaler #used for 'Feature Scaling'

#plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Prep data

In [2]:
#read dummy df
X = pd.read_csv('trip_dummies_df.csv', index_col=0)

In [3]:
X.head()

,trip_id,uniqueid,number_of_trips_taken,qtemphigh,qtemplow,qprecipitation,allwt,trip_id_count,qday_Friday,qday_Monday,qday_Saturday,qday_Sunday,qday_Thursday,qday_Tuesday,qday_Wednesday,qday1typical_Don't know,qday1typical_Not at all typical,qday1typical_Not very typical,qday1typical_Refused,qday1typical_Somewhat typical,qday1typical_Very typical,qborough_home_Brooklyn,qborough_home_Manhattan,qborough_home_Queens,qborough_home_Staten Island,qborough_home_The Bronx,qsurveyzone_home_Inner Brooklyn,qsurveyzone_home_Inner Queens,qsurveyzone_home_Manhattan Core,qsurveyzone_home_Middle Queens,qsurveyzone_home_Northern Bronx,qsurveyzone_home_Northern Manhattan,qsurveyzone_home_Outer Brooklyn,qsurveyzone_home_Outer Queens,qsurveyzone_home_Southern Bronx,qsurveyzone_home_Staten Island,qtripdaytime_AM,qtripdaytime_NOON,qtripdaytime_PM,qborough_start_Brooklyn,qborough_start_Don't know,qborough_start_Inner Brooklyn,qborough_start_Inner Queens,qborough_start_Manhattan,qborough_start_Manhattan Core,qborough_start_Middle Queens,qborough_start_Northern Bronx,qborough_start_Northern Manhattan,qborough_start_Outer Brooklyn,qborough_start_Outer Queens,qborough_start_Outside of NYC,qborough_start_Queens,qborough_start_Refused,qborough_start_Southern Bronx,qborough_start_Staten Island,qborough_start_The Bronx,surveyzone_start_Inner Brooklyn,surveyzone_start_Inner Queens,surveyzone_start_Manhattan Core,surveyzone_start_Middle Queens,surveyzone_start_NOT CODED,surveyzone_start_Northern Bronx,surveyzone_start_Northern Manhattan,surveyzone_start_Outer Brooklyn,surveyzone_start_Outer Queens,surveyzone_start_Southern Bronx,surveyzone_start_Staten Island,qday1tripend_Child's daycare facility or school,qday1tripend_Doctor's office or hospital,"qday1tripend_Entertainment event (i.e. sporting event, play, etc.)",qday1tripend_Friend or family member's home,"qday1tripend_Grocery store or market (including deli, bodega, etc.)",qday1tripend_Home,qday1tripend_Other,qday1tripend_Outside of New York City,qday1tripend_Park/Recreational area/Gym,qday1tripend_Refused,qday1tripend_Restaurant or bar,"qday1tripend_Retail store (e.g. clothing, electronic, hardware, etc.)",qday1tripend_School,qday1tripend_Work,qborough_end_Brooklyn,qborough_end_Don't know,qborough_end_Manhattan,qborough_end_Outside of NYC,qborough_end_Queens,qborough_end_Refused,qborough_end_Staten Island,qborough_end_The Bronx,surveyzone_end_Inner Brooklyn,surveyzone_end_Inner Queens,surveyzone_end_Manhattan Core,surveyzone_end_Middle Queens,surveyzone_end_Northern Bronx,surveyzone_end_Northern Manhattan,surveyzone_end_Outer Brooklyn,surveyzone_end_Outer Queens,surveyzone_end_Southern Bronx,surveyzone_end_Staten Island,qday1tripendcode_Airport,qday1tripendcode_Bus stop,qday1tripendcode_Child's daycare facility or school,qday1tripendcode_Commuter rail station,qday1tripendcode_Doctor's office or hospital,"qday1tripendcode_Entertainment event (i.e. sporting event, play, etc.)",qday1tripendcode_Friend or family member's home,qday1tripendcode_Grand Central Station,"qday1tripendcode_Grocery store or market (including deli, bodega, etc.)",qday1tripendcode_Home,qday1tripendcode_Other,qday1tripendcode_PATH Station,qday1tripendcode_Park and ride/parking lot,qday1tripendcode_Park/Recreational area/Gym,qday1tripendcode_Penn Station,qday1tripendcode_Refused,qday1tripendcode_Restaurant or bar,"qday1tripendcode_Retail store (e.g. clothing, electronic, hardware, etc.)",qday1tripendcode_Road/tunnel/bridge,qday1tripendcode_School,qday1tripendcode_Work,qday1triplength_cat_0 to 5,qday1triplength_cat_11 to 15,qday1triplength_cat_16 to 20,qday1triplength_cat_180+,qday1triplength_cat_21 to 25,qday1triplength_cat_26 to 30,qday1triplength_cat_31 to 35,qday1triplength_cat_36 to 40,qday1triplength_cat_41 to 45,qday1triplength_cat_46 to 50,qday1triplength_cat_51 to 55,qday1triplength_cat_56 to 60,qday1triplength_cat_6 to 10,qday1triplength_cat_61 to 180,qgender_Female,qgender_Male,qgender_Refused,"qincome_$100,000 - $149,999","qinc

In [4]:
#scale numerical variables
numer = X[["number_of_trips_taken", "qtemphigh", "qtemplow", "qprecipitation"]]

In [5]:
#consider removing demographic variables unrelated to mobility, refused, don't know variables

categ = X[["qday_Friday","qday_Monday","qday_Saturday","qday_Sunday","qday_Thursday","qday_Tuesday",
           "qday_Wednesday","qday1typical_Not at all typical",
           "qday1typical_Not very typical","qday1typical_Somewhat typical",
           "qday1typical_Very typical","qborough_home_Brooklyn","qborough_home_Manhattan",
           "qborough_home_Queens","qborough_home_Staten Island","qborough_home_The Bronx",
           "qsurveyzone_home_Inner Brooklyn","qsurveyzone_home_Inner Queens",
           "qsurveyzone_home_Manhattan Core","qsurveyzone_home_Middle Queens",
           "qsurveyzone_home_Northern Bronx","qsurveyzone_home_Northern Manhattan",
           "qsurveyzone_home_Outer Brooklyn","qsurveyzone_home_Outer Queens",
           "qsurveyzone_home_Southern Bronx", "qsurveyzone_home_Staten Island",
           "qtripdaytime_AM","qtripdaytime_NOON","qtripdaytime_PM","qborough_start_Brooklyn",
           "qborough_start_Inner Brooklyn",
           "qborough_start_Inner Queens","qborough_start_Manhattan","qborough_start_Manhattan Core",
           "qborough_start_Middle Queens","qborough_start_Northern Bronx","qborough_start_Northern Manhattan",
           "qborough_start_Outer Brooklyn","qborough_start_Outer Queens","qborough_start_Outside of NYC","qborough_start_Queens",
           "qborough_start_Southern Bronx","qborough_start_Staten Island",
           "qborough_start_The Bronx","surveyzone_start_Inner Brooklyn","surveyzone_start_Inner Queens",
           "surveyzone_start_Manhattan Core","surveyzone_start_Middle Queens","surveyzone_start_NOT CODED","surveyzone_start_Northern Bronx",
           "surveyzone_start_Northern Manhattan","surveyzone_start_Outer Brooklyn","surveyzone_start_Outer Queens",
           "surveyzone_start_Southern Bronx","surveyzone_start_Staten Island","qday1tripend_Child's daycare facility or school",
           "qday1tripend_Doctor's office or hospital","qday1tripend_Entertainment event (i.e. sporting event, play, etc.)",
           "qday1tripend_Friend or family member's home","qday1tripend_Grocery store or market (including deli, bodega, etc.)","qday1tripend_Home",
           "qday1tripend_Other","qday1tripend_Outside of New York City","qday1tripend_Park/Recreational area/Gym","qday1tripend_Restaurant or bar",
           "qday1tripend_Retail store (e.g. clothing, electronic, hardware, etc.)",
           "qday1tripend_School","qday1tripend_Work","qborough_end_Brooklyn",
           "qborough_end_Manhattan","qborough_end_Outside of NYC","qborough_end_Queens",
           "qborough_end_Staten Island","qborough_end_The Bronx",
           "surveyzone_end_Inner Brooklyn","surveyzone_end_Inner Queens",
           "surveyzone_end_Manhattan Core","surveyzone_end_Middle Queens",
           "surveyzone_end_Northern Bronx","surveyzone_end_Northern Manhattan",
           "surveyzone_end_Outer Brooklyn","surveyzone_end_Outer Queens",
           "surveyzone_end_Southern Bronx","surveyzone_end_Staten Island","qday1tripendcode_Airport",
           "qday1tripendcode_Bus stop","qday1tripendcode_Child's daycare facility or school",
           "qday1tripendcode_Commuter rail station","qday1tripendcode_Doctor's office or hospital",
           "qday1tripendcode_Entertainment event (i.e. sporting event, play, etc.)",
           "qday1tripendcode_Friend or family member's home","qday1tripendcode_Grand Central Station",
           "qday1tripendcode_Grocery store or market (including deli, bodega, etc.)",
           "qday1tripendcode_Home","qday1tripendcode_Other","qday1tripendcode_PATH Station","qday1tripendcode_Park and ride/parking lot",
           "qday1tripendcode_Park/Recreational area/Gym","qday1tripendcode_Penn Station",
           "qday1tripendcode_Restaurant or bar","qday1tripendcode_Retail store (e.g. clothing, electronic, hardware, etc.)",
           "qday1tripendcode_Road/tunnel/bridge","qday1tripendcode_School","qday1tripendcode_Work",
           "qday1triplength_cat_0 to 5","qday1triplength_cat_11 to 15","qday1triplength_cat_16 to 20",
           "qday1triplength_cat_180+","qday1triplength_cat_21 to 25","qday1triplength_cat_26 to 30",
           "qday1triplength_cat_31 to 35","qday1triplength_cat_36 to 40","qday1triplength_cat_41 to 45",
           "qday1triplength_cat_46 to 50","qday1triplength_cat_51 to 55","qday1triplength_cat_56 to 60",
           "qday1triplength_cat_6 to 10","qday1triplength_cat_61 to 180","qdisability1_No","qdisability1_Yes","qdisability2_No",
           "qdisability2_Yes","qdisability3_No","qdisability3_Yes","qdisability4_No","qdisability4_Yes",
           "qdisability5_No","qdisability5_Yes","qdisability6_No","qdisability6_Yes","qdisability7_No",
           "qdisability7_Yes","qdisability8_No","qdisability8_Yes","qdisability9_No","qdisability9_Yes","qagecode_18-24","qagecode_25-34",
           "qagecode_35-44","qagecode_45-54","qagecode_55-64","qagecode_65 or older",
           "qlicense_No","qlicense_Yes","qcaraccess_I do not have access to a car",
           "qcaraccess_I do not personally own or lease a car, but I have access to a car belonging to a member of my household",
           "qcaraccess_I personally own or lease a car","qcaraccess_Other","qwelfare1_No",
           "qwelfare1_Yes","qwelfare2_No","qwelfare2_Yes","qwelfare3_No","qwelfare3_Yes","qwelfare4_No",
           "qwelfare4_Yes","qwelfare5_No","qwelfare5_Yes","qcitibike_No","qcitibike_Yes","qday1triptravelcode_sp_Car Service",
           "qday1triptravelcode_sp_Carpool","qday1triptravelcode_sp_Carshare","qday1triptravelcode_sp_Citi Bike",
           "qday1triptravelcode_sp_Community van/dollar van","qday1triptravelcode_sp_Commuter rail",
           "qday1triptravelcode_sp_Electric bicycle","qday1triptravelcode_sp_Express bus","qday1triptravelcode_sp_Green taxi",
           "qday1triptravelcode_sp_Local bus","qday1triptravelcode_sp_Motorcycle","qday1triptravelcode_sp_Other",
           "qday1triptravelcode_sp_Other ferry","qday1triptravelcode_sp_PATH train","qday1triptravelcode_sp_Paratransit/ Access-A-Ride",
           "qday1triptravelcode_sp_Personal bicycle","qday1triptravelcode_sp_Personal car",
           "qday1triptravelcode_sp_Ride-hail service such as Uber or Lyft","qday1triptravelcode_sp_Select bus service",
           "qday1triptravelcode_sp_Shared-ride service such a Uber Pool, Via, or Lyft Line",
           "qday1triptravelcode_sp_Staten Island ferry","qday1triptravelcode_sp_Subway","qday1triptravelcode_sp_Walk",
           "qday1triptravelcode_sp_Yellow taxi","qpurposerecode_Accompanying other traveler",
           "qpurposerecode_Business","qpurposerecode_Commute to/from work","qpurposerecode_Dining",
           "qpurposerecode_Medical visit (doctor's office)","qpurposerecode_Other","qpurposerecode_Personal errands",
           "qpurposerecode_School","qpurposerecode_Shopping","qpurposerecode_Social/recreation"]]




In [6]:
#initialize scalar
scaler = StandardScaler()

In [7]:
#scale numerical data
numer = pd.DataFrame(scaler.fit_transform(numer))

In [8]:
#rename columns
numer.columns = ["number_of_trips_taken", "qtemphigh", "qtemplow", "qprecipitation"]

In [9]:
X = pd.concat([numer, categ], axis=1, join='inner')

In [10]:
X.head()

,number_of_trips_taken,qtemphigh,qtemplow,qprecipitation,qday_Friday,qday_Monday,qday_Saturday,qday_Sunday,qday_Thursday,qday_Tuesday,qday_Wednesday,qday1typical_Not at all typical,qday1typical_Not very typical,qday1typical_Somewhat typical,qday1typical_Very typical,qborough_home_Brooklyn,qborough_home_Manhattan,qborough_home_Queens,qborough_home_Staten Island,qborough_home_The Bronx,qsurveyzone_home_Inner Brooklyn,qsurveyzone_home_Inner Queens,qsurveyzone_home_Manhattan Core,qsurveyzone_home_Middle Queens,qsurveyzone_home_Northern Bronx,qsurveyzone_home_Northern Manhattan,qsurveyzone_home_Outer Brooklyn,qsurveyzone_home_Outer Queens,qsurveyzone_home_Southern Bronx,qsurveyzone_home_Staten Island,qtripdaytime_AM,qtripdaytime_NOON,qtripdaytime_PM,qborough_start_Brooklyn,qborough_start_Inner Brooklyn,qborough_start_Inner Queens,qborough_start_Manhattan,qborough_start_Manhattan Core,qborough_start_Middle Queens,qborough_start_Northern Bronx,qborough_start_Northern Manhattan,qborough_start_Outer Brooklyn,qborough_start_Outer Queens,qborough_start_Outside of NYC,qborough_start_Queens,qborough_start_Southern Bronx,qborough_start_Staten Island,qborough_start_The Bronx,surveyzone_start_Inner Brooklyn,surveyzone_start_Inner Queens,surveyzone_start_Manhattan Core,surveyzone_start_Middle Queens,surveyzone_start_NOT CODED,surveyzone_start_Northern Bronx,surveyzone_start_Northern Manhattan,surveyzone_start_Outer Brooklyn,surveyzone_start_Outer Queens,surveyzone_start_Southern Bronx,surveyzone_start_Staten Island,qday1tripend_Child's daycare facility or school,qday1tripend_Doctor's office or hospital,"qday1tripend_Entertainment event (i.e. sporting event, play, etc.)",qday1tripend_Friend or family member's home,"qday1tripend_Grocery store or market (including deli, bodega, etc.)",qday1tripend_Home,qday1tripend_Other,qday1tripend_Outside of New York City,qday1tripend_Park/Recreational area/Gym,qday1tripend_Restaurant or bar,"qday1tripend_Retail store (e.g. clothing, electronic, hardware, etc.)",qday1tripend_School,qday1tripend_Work,qborough_end_Brooklyn,qborough_end_Manhattan,qborough_end_Outside of NYC,qborough_end_Queens,qborough_end_Staten Island,qborough_end_The Bronx,surveyzone_end_Inner Brooklyn,surveyzone_end_Inner Queens,surveyzone_end_Manhattan Core,surveyzone_end_Middle Queens,surveyzone_end_Northern Bronx,surveyzone_end_Northern Manhattan,surveyzone_end_Outer Brooklyn,surveyzone_end_Outer Queens,surveyzone_end_Southern Bronx,surveyzone_end_Staten Island,qday1tripendcode_Airport,qday1tripendcode_Bus stop,qday1tripendcode_Child's daycare facility or school,qday1tripendcode_Commuter rail station,qday1tripendcode_Doctor's office or hospital,"qday1tripendcode_Entertainment event (i.e. sporting event, play, etc.)",qday1tripendcode_Friend or family member's home,qday1tripendcode_Grand Central Station,"qday1tripendcode_Grocery store or market (including deli, bodega, etc.)",qday1tripendcode_Home,qday1tripendcode_Other,qday1tripendcode_PATH Station,qday1tripendcode_Park and ride/parking lot,qday1tripendcode_Park/Recreational area/Gym,qday1tripendcode_Penn Station,qday1tripendcode_Restaurant or bar,"qday1tripendcode_Retail store (e.g. clothing, electronic, hardware, etc.)",qday1tripendcode_Road/tunnel/bridge,qday1tripendcode_School,qday1tripendcode_Work,qday1triplength_cat_0 to 5,qday1triplength_cat_11 to 15,qday1triplength_cat_16 to 20,qday1triplength_cat_180+,qday1triplength_cat_21 to 25,qday1triplength_cat_26 to 30,qday1triplength_cat_31 to 35,qday1triplength_cat_36 to 40,qday1triplength_cat_41 to 45,qday1triplength_cat_46 to 50,qday1triplength_cat_51 to 55,qday1triplength_cat_56 to 60,qday1triplength_cat_6 to 10,qday1triplength_cat_61 to 180,qdisability1_No,qdisability1_Yes,qdisability2_No,qdisability2_Yes,qdisability3_No,qdisability3_Yes,qdisability4_No,qdisability4_Yes,qdisability5_No,qdisability5_Yes,qdisability6_No,qdisability6_Yes,qdisability7_No,qdisability7_Yes,qdisability8_No,qdisability8_Yes,qdisability9_No,qdisability9_Yes,qage

In [11]:
X.to_csv('scaled_dummy_reduced_df.csv')

In [12]:
#removed features
# "qincome_ 100,000− 149,999","qincome_ 15,000− 24,999","qincome_ 150,000− 199,999","qincome_$200,000 and above",
#            "qincome_ 25,000− 34,999","qincome_ 35,000− 49,999","qincome_ 50,000− 74,999","qincome_ 75,000− 99,999",
#            "qincome_Less than $14,999","qincome_Refused",,

# "qracecode_Asian, Non-Hispanic",
#            "qracecode_Black, Non-Hispanic","qracecode_Don't know, Non-Hispanic","qracecode_Hispanic",
#            "qracecode_Other, Non-Hispanic","qracecode_White, Non-Hispanic","qeducation_Associate degree (i.e., AA, AS)",
#            "qeducation_Bachelor's degree (i.e., BA, BS, AB)","qeducation_Graduate degree (i.e., Master's, Professional, Doctorate)",
#            "qeducation_High school graduate or equivalent (i.e., GED)","qeducation_No high school",
#            "qeducation_Some college but degree not received or in progress","qeducation_Some high school",

# "qsmartphone_No","qsmartphone_Refused","qsmartphone_Yes",
#            "qshare1_No","qshare1_Yes","qshare2_No","qshare2_Yes","qshare3_No","qshare3_Yes","qshare4_No",
#            "qshare4_Yes","qshare5_No","qshare5_Yes","qshare6_No","qshare6_Yes","qshare7_No","qshare7_Yes",
#            "qshare8_No","qshare8_Yes",

# "qgender_Female","qgender_Male",
#            "qgender_Refused",

# "qday1typical_Don't know",
# "qborough_start_Don't know",
# "qborough_end_Don't know",
# "qday1triptravelcode_sp_Don't know",
# "qday1typical_Refused",
# "qborough_start_Refused",
# "qday1tripend_Refused",
# "qborough_end_Refused",
# "qlicense_Refused",
# "qday1tripendcode_Refused",
# "qpurposerecode_Refused",
# "qcitibike_Refused",
# "qday1triptravelcode_sp_Refused",
# "qcaraccess_Refused",


# #IMPORTANT: REMOVED TRANSIT MODE
# "qmodegrouping_Bike","qmodegrouping_Bus","qmodegrouping_Car",
#            "qmodegrouping_Commuter Rail","qmodegrouping_Ferry","qmodegrouping_For-Hire Vehicle","qmodegrouping_Other",
#            "qmodegrouping_Subway","qmodegrouping_Walk","qsustainablemode_No","qsustainablemode_Yes",


# Cluster!

In [13]:
#Initialize model
kmeans = KMeans(n_clusters=3)

In [14]:
#fit model
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [15]:
#Find which cluster each data-point belongs to
clusters = kmeans.predict(X)

In [16]:
#add cluster vector to X df
X["Cluster"] = clusters


In [19]:
X.head()

,number_of_trips_taken,qtemphigh,qtemplow,qprecipitation,qday_Friday,qday_Monday,qday_Saturday,qday_Sunday,qday_Thursday,qday_Tuesday,qday_Wednesday,qday1typical_Not at all typical,qday1typical_Not very typical,qday1typical_Somewhat typical,qday1typical_Very typical,qborough_home_Brooklyn,qborough_home_Manhattan,qborough_home_Queens,qborough_home_Staten Island,qborough_home_The Bronx,qsurveyzone_home_Inner Brooklyn,qsurveyzone_home_Inner Queens,qsurveyzone_home_Manhattan Core,qsurveyzone_home_Middle Queens,qsurveyzone_home_Northern Bronx,qsurveyzone_home_Northern Manhattan,qsurveyzone_home_Outer Brooklyn,qsurveyzone_home_Outer Queens,qsurveyzone_home_Southern Bronx,qsurveyzone_home_Staten Island,qtripdaytime_AM,qtripdaytime_NOON,qtripdaytime_PM,qborough_start_Brooklyn,qborough_start_Inner Brooklyn,qborough_start_Inner Queens,qborough_start_Manhattan,qborough_start_Manhattan Core,qborough_start_Middle Queens,qborough_start_Northern Bronx,qborough_start_Northern Manhattan,qborough_start_Outer Brooklyn,qborough_start_Outer Queens,qborough_start_Outside of NYC,qborough_start_Queens,qborough_start_Southern Bronx,qborough_start_Staten Island,qborough_start_The Bronx,surveyzone_start_Inner Brooklyn,surveyzone_start_Inner Queens,surveyzone_start_Manhattan Core,surveyzone_start_Middle Queens,surveyzone_start_NOT CODED,surveyzone_start_Northern Bronx,surveyzone_start_Northern Manhattan,surveyzone_start_Outer Brooklyn,surveyzone_start_Outer Queens,surveyzone_start_Southern Bronx,surveyzone_start_Staten Island,qday1tripend_Child's daycare facility or school,qday1tripend_Doctor's office or hospital,"qday1tripend_Entertainment event (i.e. sporting event, play, etc.)",qday1tripend_Friend or family member's home,"qday1tripend_Grocery store or market (including deli, bodega, etc.)",qday1tripend_Home,qday1tripend_Other,qday1tripend_Outside of New York City,qday1tripend_Park/Recreational area/Gym,qday1tripend_Restaurant or bar,"qday1tripend_Retail store (e.g. clothing, electronic, hardware, etc.)",qday1tripend_School,qday1tripend_Work,qborough_end_Brooklyn,qborough_end_Manhattan,qborough_end_Outside of NYC,qborough_end_Queens,qborough_end_Staten Island,qborough_end_The Bronx,surveyzone_end_Inner Brooklyn,surveyzone_end_Inner Queens,surveyzone_end_Manhattan Core,surveyzone_end_Middle Queens,surveyzone_end_Northern Bronx,surveyzone_end_Northern Manhattan,surveyzone_end_Outer Brooklyn,surveyzone_end_Outer Queens,surveyzone_end_Southern Bronx,surveyzone_end_Staten Island,qday1tripendcode_Airport,qday1tripendcode_Bus stop,qday1tripendcode_Child's daycare facility or school,qday1tripendcode_Commuter rail station,qday1tripendcode_Doctor's office or hospital,"qday1tripendcode_Entertainment event (i.e. sporting event, play, etc.)",qday1tripendcode_Friend or family member's home,qday1tripendcode_Grand Central Station,"qday1tripendcode_Grocery store or market (including deli, bodega, etc.)",qday1tripendcode_Home,qday1tripendcode_Other,qday1tripendcode_PATH Station,qday1tripendcode_Park and ride/parking lot,qday1tripendcode_Park/Recreational area/Gym,qday1tripendcode_Penn Station,qday1tripendcode_Restaurant or bar,"qday1tripendcode_Retail store (e.g. clothing, electronic, hardware, etc.)",qday1tripendcode_Road/tunnel/bridge,qday1tripendcode_School,qday1tripendcode_Work,qday1triplength_cat_0 to 5,qday1triplength_cat_11 to 15,qday1triplength_cat_16 to 20,qday1triplength_cat_180+,qday1triplength_cat_21 to 25,qday1triplength_cat_26 to 30,qday1triplength_cat_31 to 35,qday1triplength_cat_36 to 40,qday1triplength_cat_41 to 45,qday1triplength_cat_46 to 50,qday1triplength_cat_51 to 55,qday1triplength_cat_56 to 60,qday1triplength_cat_6 to 10,qday1triplength_cat_61 to 180,qdisability1_No,qdisability1_Yes,qdisability2_No,qdisability2_Yes,qdisability3_No,qdisability3_Yes,qdisability4_No,qdisability4_Yes,qdisability5_No,qdisability5_Yes,qdisability6_No,qdisability6_Yes,qdisability7_No,qdisability7_Yes,qdisability8_No,qdisability8_Yes,qdisability9_No,qdisability9_Yes,qage

In [17]:
#from learn lab: https://github.com/learn-co-students/ds-k-means-clustering-lab-nyc-ds-111819/tree/solution
from sklearn import metrics
labels = kmeans.labels_

metrics.silhouette_score(X, labels, metric='euclidean')
#output: 0.09959917567091452

0.09959917567091452

Try different # clusters

In [42]:
kmeans2 = KMeans(n_clusters = 2)
kmeans2.fit(X)
clusters2 = kmeans2.predict(X)

In [43]:
labels = kmeans2.labels_

metrics.silhouette_score(X, labels, metric='euclidean')
# n = 2 : 0.3004255254012257
# n = 3: 0.09959917567091452
# n =12 : 0.12104025744036882
# n = 13 : 0.1243551046123975
# n = 14 : 0.10297305268073777
# n=15 :0.10595328684969835
# n=100 : 0.055124400116253136


0.3004255254012257

Calinski-Harabaz Score

In [44]:
#Calinski_Harabasz for 3 clusters
metrics.calinski_harabasz_score(X, labels) 

1433.249193800052

In [45]:
#add cluster vector to X df
X["Cluster"] = clusters2

# PCA

In [46]:
#PCA with one principal component
pca_1d = PCA(n_components=1)

#PCA with two principal components
pca_2d = PCA(n_components=2)

#PCA with three principal components
pca_3d = PCA(n_components=3)

In [47]:
#This DataFrame holds that single principal component mentioned above
PCs_1d = pd.DataFrame(pca_1d.fit_transform(X.drop(["Cluster"], axis=1)))

#This DataFrame contains the two principal components that will be used
#for the 2-D visualization mentioned above
PCs_2d = pd.DataFrame(pca_2d.fit_transform(X.drop(["Cluster"], axis=1)))

#And this DataFrame contains three principal components that will aid us
#in visualizing our clusters in 3-D
PCs_3d = pd.DataFrame(pca_3d.fit_transform(X.drop(["Cluster"], axis=1)))

In [48]:
PCs_1d.columns = ["PC1_1d"]

#"PC1_2d" means: 'The first principal component of the components created for 2-D visualization, by PCA.'
#And "PC2_2d" means: 'The second principal component of the components created for 2-D visualization, by PCA.'
PCs_2d.columns = ["PC1_2d", "PC2_2d"]

PCs_3d.columns = ["PC1_3d", "PC2_3d", "PC3_3d"]

In [49]:
# We concatenate these newly created DataFrames to X so 
# that they can be used by X as columns.
X = pd.concat([X,PCs_1d,PCs_2d,PCs_3d], axis=1, join='inner')

In [50]:
# And we create one new column for plotX so that we can use it for
# 1-D visualization.
X["dummy"] = 0

In [51]:
# Each of these new DataFrames will hold all of the values contained
# in exacltly one of the clusters. For example, all of the values
# contained within the DataFrame, cluster0 will belong to 'cluster 0',
# and all the values contained in DataFrame, cluster1 will belong to
# 'cluster 1', etc.
#Note that all of the DataFrames below are sub-DataFrames of 'plotX'.
#This is because we intend to plot the values contained within each of these DataFrames.

cluster0 = X[X["Cluster"] == 0]
cluster1 = X[X["Cluster"] == 1]
#cluster2 = X[X["Cluster"] == 2]

In [52]:
#This is needed so we can display plotly plots properly
init_notebook_mode(connected=True)

In [53]:
#Instructions for building the 1-D plot

#trace1 is for 'Cluster 0'
trace1 = go.Scatter(
                    x = cluster0["PC1_1d"],
                    y = cluster0["dummy"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 is for 'Cluster 1'
trace2 = go.Scatter(
                    x = cluster1["PC1_1d"],
                    y = cluster1["dummy"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

# #trace3 is for 'Cluster 2'
# trace3 = go.Scatter(
#                     x = cluster2["PC1_1d"],
#                     y = cluster2["dummy"],
#                     mode = "markers",
#                     name = "Cluster 2",
#                     marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
#                     text = None)

data = [trace1, trace2] #trace3

title = "Visualizing Clusters in One Dimension Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= '',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [54]:
# Instructions for building the 2-D plot

#trace1 is for 'Cluster 0'
trace1 = go.Scatter(
                    x = cluster0["PC1_2d"],
                    y = cluster0["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 is for 'Cluster 1'
trace2 = go.Scatter(
                    x = cluster1["PC1_2d"],
                    y = cluster1["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

# #trace3 is for 'Cluster 2'
# trace3 = go.Scatter(
#                     x = cluster2["PC1_2d"],
#                     y = cluster2["PC2_2d"],
#                     mode = "markers",
#                     name = "Cluster 2",
#                     marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
#                     text = None)

data = [trace1, trace2] #, trace3

title = "Visualizing Clusters in Two Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [55]:
#Instructions for building the 3-D plot

#trace1 is for 'Cluster 0'
trace1 = go.Scatter3d(
                    x = cluster0["PC1_3d"],
                    y = cluster0["PC2_3d"],
                    z = cluster0["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 is for 'Cluster 1'
trace2 = go.Scatter3d(
                    x = cluster1["PC1_3d"],
                    y = cluster1["PC2_3d"],
                    z = cluster1["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

# #trace3 is for 'Cluster 2'
# trace3 = go.Scatter3d(
#                     x = cluster2["PC1_3d"],
#                     y = cluster2["PC2_3d"],
#                     z = cluster2["PC3_3d"],
#                     mode = "markers",
#                     name = "Cluster 2",
#                     marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
#                     text = None)

data = [trace1, trace2] #, trace3

title = "Clusters with 3 Principal Components"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)